# LSTM Networks
An LSTM Network, short for Long Short-Term Memory Network, is a type of recurrent neural network that can remember from timesteps long ago. A classic example of a classic RNN's shortcomings are predicting the next word in the sequence: I am French. (2000 words later) I speak fluent <i>French</i>. A classic RNN might predict any language, as it only remembers short term due to the vanishing gradient problem: since the hidden layer is constantly being multiplied by a value less than one, so the memory from timesteps deeper in the past are exponentially less significant.

However, an LSTM network retains this memory through the use of a cell state: where long term memory is stored. Their ethnicity would be stored in the cell state, and it would predict the correct word. An LSTM's parameters tell it what to forget by element-wise multiplication, new things to remember by addition, and what parts of memory to select and pass on to the output of an LSTM cell, which, amazingly, can be learned effectively by performing an optimization technique such as RMSProp.

These are done by gates, represented by rounded rectangles and their corresponding activation, including input (i), forget (f), input modulation (c tilde), and output (o) gates. Each gate is just a perceptron: with its own weights and biases. This LSTM has two weight matrices per gate: one for the input x, and another for the hidden state of the last layer. The hidden state, or output of the LSTM cell, is then passed into the final layer of the network, which will be the final output.

This LSTM network is character-based: it tries to predict the next character in a sequence, where each character is represented by a 78 dimensional 1hot vector.

![](http://www.mdpi.com/energies/energies-10-01168/article_deploy/html/images/energies-10-01168-g008.png)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/2db2cba6a0d878e13932fa27ce6f3fb71ad99cf1)

In [ ]:
import numpy as np

In [ ]:
def sigmoid(x, deriv=False):
    if not deriv:
        return 1 / (1 + np.exp(-x))
    else:
        return x * (1 - x)
    
def tanh(x, deriv=False):
    if not deriv:
        return np.tanh(x)
    else:
        return 1 - (x **2)
    
# Softmax function for output probabilities
def softmax(x):
    nonlin_x = np.exp(x)
    return nonlin_x / np.sum(nonlin_x, 1).reshape(nonlin_x.shape[0], 1)

In [ ]:
class LSTM:
    def __init__(self, learning_rate, input_dim, hidden_dim, output_dim):
        self.learning_rate = learning_rate
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Initializing weight matrices
        self.Wf = np.random.randn(input_dim, hidden_dim) * 0.01
        self.Wi = np.random.randn(input_dim, hidden_dim) * 0.01
        self.Wj = np.random.randn(input_dim, hidden_dim) * 0.01
        self.Wo = np.random.randn(input_dim, hidden_dim) * 0.01
        
        # Initializing hidden state weight matrices
        self.Uf = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.Ui = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.Uj = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.Uo = np.random.randn(hidden_dim, hidden_dim) * 0.01
        
        # Initializing biases
        self.Bf = np.random.randn(1, hidden_dim) * 0.01
        self.Bi = np.random.randn(1, hidden_dim) * 0.01
        self.Bj = np.random.randn(1, hidden_dim) * 0.01
        self.Bo = np.random.randn(1, hidden_dim) * 0.01
        
        # Final layer parameters
        self.W2 = np.random.randn(hidden_dim, output_dim) * 0.01
        self.B2 = np.random.randn(1, output_dim) * 0.01
        
        # Memory: used in RMSProp optimizer
        self.memory = [np.zeros([input_dim, hidden_dim]),
                               np.zeros([input_dim, hidden_dim]), 
                               np.zeros([input_dim, hidden_dim]),
                               np.zeros([input_dim, hidden_dim]),
                               np.zeros([hidden_dim, output_dim]),
                               np.zeros([1, hidden_dim]),
                               np.zeros([1, hidden_dim]),
                               np.zeros([1, hidden_dim]),
                               np.zeros([1, hidden_dim]),
                               np.zeros([1, output_dim]),
                               np.zeros([hidden_dim, hidden_dim]), 
                               np.zeros([hidden_dim, hidden_dim]),
                               np.zeros([hidden_dim, hidden_dim]),
                               np.zeros([hidden_dim, hidden_dim])]
        
    # Getting a generated sample by predicting the next in a sequence
    # over and over again, and adding the prediction to the sequence.
    def sample(self, h_prev, c_prev, startchar, length=200):
        Xs = [startchar]
        prev_char = startchar
        
        # Getting previous cell and hidden states:
        Cp = c_prev
        Hp = h_prev
        
        for t in range(length):
            # Forward pass
            
            X = np.zeros([1, self.input_dim])
            X[:,prev_char] = 1
            
            # Gates
            pf = np.dot(X, self.Wf) + np.dot(Hp, self.Uf) + self.Bf
            f = sigmoid(pf)
            pi = np.dot(X, self.Wi) + np.dot(Hp, self.Ui) + self.Bi
            i = sigmoid(pi)
            pj = np.dot(X, self.Wj) + np.dot(Hp, self.Uj) + self.Bj
            j = tanh(pj)
            po = np.dot(X, self.Wo) + np.dot(Hp, self.Uo) + self.Bo
            o = sigmoid(po)
            
            # Cell state and hidden state operations
            c = (Cp * f) + (i * j)
            thc = tanh(c)
            h = o * thc
            Cp = c
            Hp = h
            
            # Final layer logits
            pyh = np.dot(h, self.W2) + self.B2
            # Selecting the most probable outcome to be the
            # next in the sequence
            new_x = np.argmax(pyh)
            
            Xs.append(new_x)
            prev_char = new_x
            
        return Xs
       
    # Computing gradients for the network
    def compute_gradients(self, X, Y, h_prev, c_prev):
        # Initialize cell and hidden states from the previous LSTM cell
        # Since there is no previous cell, initialize them as 0
        Cp = c_prev
        Hp = h_prev
        
        # Error for keeping track of progress
        error = 0
        
        # Initializing values inside the LSTM cell at every timestep that
        # are necessary for computing gradients
        Yh,H,C,O,I,J,F = [],[],[],[],[],[],[]
        
        # Forward pass: forward through time
        for t in range(len(X)):
            # Get gating values: J in this code represents the input modulation gate,
            # or C tilde in the diagram above
            f = sigmoid(np.dot(X[t:t+1], self.Wf) + np.dot(Hp, self.Uf) + self.Bf)
            i = sigmoid(np.dot(X[t:t+1], self.Wi) + np.dot(Hp, self.Ui) + self.Bi)
            j = tanh(np.dot(X[t:t+1], self.Wj) + np.dot(Hp, self.Uj) + self.Bj)
            o = sigmoid(np.dot(X[t:t+1], self.Wo) + np.dot(Hp, self.Uo) + self.Bo)
            
            # Multiply the forget by the previous cell state and add the input:
            # input gate * input modulation gate
            c = (Cp * f) + (i * j)
            # Hidden state: multiplying by output gate, selecting what to output
            # from the LSTM cell
            h = o * tanh(c)
            # Setting states to current states for use in the next timestep
            Cp = c
            Hp = h
            
            # Passing hidden state through final layer producing output probabilities
            pyh = np.dot(h, self.W2) + self.B2
            yh = softmax(pyh)
            
            # Crossentropy error, not necessary, however, for computing gradients
            error -= np.sum(Y[t] * np.log(yh))
            
            # Append values from forward pass for use in backpropagation and computing
            # gradients
            Yh.append(yh)
            H.append(h)
            C.append(c)
            O.append(o)
            I.append(i)
            J.append(j)
            F.append(f)
        
        # Backpropagation time: initialize derivatives of gate pre-activations as 0
        dPF = np.zeros([1, self.hidden_dim])
        dPI = np.zeros([1, self.hidden_dim])
        dPJ = np.zeros([1, self.hidden_dim])
        dPO = np.zeros([1, self.hidden_dim])
        
        # Initialize derivative of cell state with respect to previous cell state as 0
        dCC = np.zeros([1, self.hidden_dim])
        
        # Initialize parameter gradients as 0
        Wf_grad, Wi_grad, Wj_grad, Wo_grad, W2_grad = 0, 0, 0, 0, 0
        Bf_grad, Bi_grad, Bj_grad, Bo_grad, B2_grad = 0, 0, 0, 0, 0
        Uf_grad, Ui_grad, Uj_grad, Uo_grad = 0, 0, 0, 0
        
        # Backward pass through time
        for t in reversed(range(len(X))):
            # Deriv of crossentropy w.r.t. logits
            dPYh = Yh[t] - Y[t]
            
            # Derivative of hidden state: it affects the logits at
            # the current timestep and all the gating values of the cell
            # at the next timestep, all affecting the total error
            dH = (np.dot(dPYh, self.W2.T) + np.dot(dPF, self.Uf.T) + np.dot(dPI, self.Ui.T) + 
                  np.dot(dPJ, self.Uj.T) + np.dot(dPO, self.Uo.T))
            
            # Derivative of cell state: affects outcome of hidden state, as
            # well as next cell state, which affects the next hidden state,
            # and so on, represented by dCC: derivative of cell state with 
            # respect to previous cell state
            dC = dH * O[t] * tanh(tanh(C[t]), deriv=True) + dCC
            
            # derivative of cell state with respect to previous cell state is the
            # derivative of cell state * forget: using the chain rule through
            # the formula for computing the next cell state
            dCC = dC * F[t]
            
            # Use deriv of cell state to compute derivs of gate pre-activations
            
            if t != 0:
                dPF = dC * C[t - 1] * sigmoid(F[t], deriv=True)
            else:
                dPF = np.zeros([1, hidden_dim])
                
            dPI = dC * J[t] * sigmoid(I[t], deriv=True)
            dPJ = dC * I[t] * tanh(J[t], deriv=True)
            
            # Use deriv of hidden state to compute deriv of output gate
            # pre-activations
            dPO = dH * tanh(C[t]) * sigmoid(O[t], deriv=True)
            
            # Using deriv of gate pre-activations to compute weight gradients
            Wo_grad += np.dot(X[t:t+1].T, dPO)
            Wj_grad += np.dot(X[t:t+1].T, dPJ)
            Wi_grad += np.dot(X[t:t+1].T, dPI)
            Wf_grad += np.dot(X[t:t+1].T, dPF)
            
            # Likewise, with bias gradients
            Bo_grad += dPO
            Bj_grad += dPJ
            Bi_grad += dPI
            Bf_grad += dPF
            
            # Likewise, with hidden state to gate matrices
            if t != 0:
                Uo_grad += np.dot(H[t - 1].T, dPO)
                Uj_grad += np.dot(H[t - 1].T, dPJ)
                Ui_grad += np.dot(H[t - 1].T, dPI)
                Uf_grad += np.dot(H[t - 1].T, dPF)
                
            # Compute gradients of final layer
            W2_grad += np.dot(H[t].T, dPYh)
            B2_grad += dPYh
            
        gradients = []
            
        # Normalize gradients by dividing by sequence length and
        # store them in list
        for theta in (Wf_grad, Wi_grad, Wj_grad, Wo_grad, W2_grad, Bf_grad, Bi_grad, Bj_grad, Bo_grad, B2_grad,
                      Uf_grad, Ui_grad, Uj_grad, Uo_grad):
            gradients.append(theta / len(X))
            
        # Return normalized error and gradients
        return error / len(X), tuple(gradients), Hp, Cp
    
    # Applying RMSProp: an optimization strategy to adjust weights and 
    # biases similar to Adagrad, without decaying the learning rate 
    # significantly more than necessary
    def apply_rmsprop(self, gradients, gamma):
        
        # Set params to list of all weights and biases
        theta = [self.Wf, self.Wi, self.Wj, self.Wo, self.W2, self.Bf, self.Bi,
                  self.Bj, self.Bo, self.B2, self.Uf, self.Ui, self.Uj, self.Uo]
        
        # Iterate through parameter matrices with memory
        # and gradients
        for i in range(len(theta)):
            
            self.memory[i] = gamma * self.memory[i] + (1 - gamma) * (gradients[i] * gradients[i])
            
            theta[i] += -self.learning_rate * gradients[i] / np.sqrt(self.memory[i] + 1e-8)

In [ ]:
# Preprocessing
text = open("eminem.txt", 'r').read()
len_text = len(text)

# Creating a list that contains every unique character
in_to_char = sorted(list(set(text)))

# Total unique characters
unique_chars = len(in_to_char)

# Dictionary that maps characters to their respective indices
char_to_in = {in_to_char[i]:i for i in range(len(in_to_char))}

# Function retrieving next sequence of data
current_in = 0
def next_batch(seq_length):
    # Update index to get different text every time
    global current_in
    if current_in + seq_length + 1 >= len_text:
        current_in = 0
    
    # Get indices, set y to the next x
    x_in = [char_to_in[x] for x in text[current_in:current_in+seq_length]]
    y_in = [char_to_in[x] for x in text[current_in+1:current_in+seq_length+1]]
    
    x_1hot = np.zeros([seq_length, unique_chars])
    y_1hot = np.zeros([seq_length, unique_chars])
    
    # Convert indices into 1hot
    for i in range(len(x_in)):
        x_1hot[i][x_in[i]] = 1
        y_1hot[i][y_in[i]] = 1
        
    current_in += seq_length
        
    return x_1hot, y_1hot

In [ ]:
seq_length = 200
learning_rate = 0.002
epochs = 60000
hidden_dim = 100
display_step = 5
gamma = 0.9

# Initializing network object
lstm = LSTM(learning_rate, unique_chars, hidden_dim, unique_chars)

h_prev, c_prev = [np.zeros([1, hidden_dim])] * 2

for epoch in range(epochs):
    # Get next sequence
    x_1hot, y_1hot = next_batch(seq_length)
    
    # Train network and update parameters after each sequence
    error, gradients, h_prev, c_prev = lstm.compute_gradients(x_1hot, y_1hot, h_prev, c_prev)
    lstm.apply_rmsprop(gradients, gamma)
    
    if epoch % display_step == 0:
        raw = lstm.sample(h_prev, c_prev, char_to_in['T'])
        print("Epoch", epoch, "Error", error)
        print("\n\n", "".join(in_to_char[i] for i in raw), "\n\n")

In [ ]:
# Generate 2000 character sample after training

sample = lstm.sample(np.zeros([1, hidden_dim]), np.zeros([1, hidden_dim]), char_to_in['I'], length=2000)
print("\n\n", "".join(in_to_char[i] for i in sample), "\n\n")